In [1]:
import os 
import pandas as pd 
import sys 
import re 

current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path,'..','..'))  # '..','..','..'
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
    
from pipeline.plotting.TS_analysis import plot_TS
from jupyter_ipynb.NetMob_training_analysis.plotting import plot_boxplot_on_metric
from experiences.convert_df_to_latex import dataframe_to_latex,parse_index_exp1_2,parse_index_exp4
from experiences.convert_df_to_latex import update_df_metrics,load_csv,results_to_dataframe
from experiences.convert_df_to_latex import tackle_trial_j as tackle_trial_for_distrib
from experiences.common_results import dic_exp_to_names,find_baseline,dic_trials
import os 
import pandas as pd 
from pipeline.plotting.TS_analysis import plot_TS

folder_path = '../../save/K_fold_validation/training_wo_HP_tuning'  # '../../save'


In [2]:
for exp_i,trials in dic_trials.items():
    if exp_i == 'set_experience_HERE' :#'Exp1_subway_in': #'set_experience_HERE': # set_experience_HERE # 'Exp1_subway_in'# 'Exp4_15min' # 'Exp4' # 'Exp3_bike_15min_h4' # 'Exp3' # 'Exp1'
        for trial_j in trials:
            df_j_all = pd.DataFrame()
            for n_bis in range(1,6):
                file_path = f"{folder_path}/{exp_i}/{dic_exp_to_names[exp_i]}/{trial_j}_bis{n_bis}/Losses_{trial_j}_bis{n_bis}.csv"
                df_j = pd.read_csv(file_path,index_col = 0)
                df_j.columns = [f'Train_{n_bis}',f'Valid_{n_bis}']
                df_j_all = pd.concat([df_j_all,df_j],axis=1)
            plot_TS(df_j_all,width=1500,height=400,bool_show=True,title=f"{exp_i}, {trial_j}",x_datetime = False)


In [6]:
# Get Data from saved csv files:  
def tackle_trial_j(folder_path,dic_exp_to_names,L_metrics,exp_i,trial_j,metrics,plot_losses):
    df_j_all = pd.DataFrame()
    metric_i = []
    for n_bis in range(1,6):
        df_j_all, metric_i = load_csv(folder_path,dic_exp_to_names,exp_i,trial_j,n_bis,df_j_all,metric_i,metrics)

    metric_i = pd.DataFrame(pd.DataFrame(metric_i).agg(['mean','std']).unstack()).T
    metric_i.index = [f"{trial_j}"]
    L_metrics.append(metric_i)
    if plot_losses:
            plot_TS(df_j_all,width=1500,height=400,bool_show=True,title=f"{exp_i}, {trial_j}",x_datetime = False)
    return L_metrics

folder_path = 'save/K_fold_validation/training_wo_HP_tuning' # ../../
metrics = ['rmse','mae','mase']
plot_losses = False 

experiences = ['Exp6_bike_netmob','Exp6_subway_netmob'] # 'Exp3','Exp3_bike_15min_h4','Exp4', 'Exp4_15min_h1','Exp4_15min'
for exp_i in experiences:
    trials = dic_trials[exp_i]
    print(f"\n------------------ {exp_i} ---")
    L_metrics = []
    for trial_j in trials:
        if (exp_i == 'Exp3_bike_15min_h4') and ('STAEformer_bike_out_calendar__e50_h4' in trial_j  or 'STAEformer_bike_out_calendar__e80_h4' in trial_j):
            exp_i_tmp = 'Exp4_15min'
        elif exp_i == 'Exp4_15min_h1':
            exp_i_tmp = 'Exp4_15min'
        else:
            exp_i_tmp = exp_i
        L_metrics = tackle_trial_j(folder_path,dic_exp_to_names,L_metrics,exp_i_tmp,trial_j,metrics,plot_losses)
        
    df_metrics_all = pd.concat(L_metrics)

    horizons = list(set([c[0].split('_')[-1][1:] for c in df_metrics_all.columns]))
    for horizon in horizons:
        df_horizon = df_metrics_all[[c for c in df_metrics_all.columns if c[0].endswith(f"_h{horizon}")]].dropna()

        globals()[f"df_metrics_{exp_i}_h{horizon}"] = df_horizon.sort_values(by=[(f'rmse_h{horizon}','mean')])

        baseline_index = find_baseline(exp_i,h=horizon)
        for c in globals()[f"df_metrics_{exp_i}_h{horizon}"].columns:
            if 'mean' == c[1]:
                globals()[f"df_metrics_{exp_i}_h{horizon}"][c] = globals()[f"df_metrics_{exp_i}_h{horizon}"][c].apply(lambda x: f"{round(x,2)} ({round(100*(-1+x/globals()[f'df_metrics_{exp_i}_h{horizon}'].loc[baseline_index,c]),2)}\%)")
        
        display(globals()[f"df_metrics_{exp_i}_h{horizon}"])


# Get data from results: 





------------------ Exp6_bike_netmob ---


rmse_h4            \
                                                              mean       std   
STAEformer_bike_out_netmob_POIs_calendar_late_f...  5.03 (-1.22\%)  0.061381   
STAEformer_bike_out_netmob_POIs_calendar_late_f...  5.07 (-0.52\%)  0.044188   
STAEformer_bike_out_netmob_POIs_calendar_late_f...   5.07 (-0.5\%)  0.039379   
STAEformer_bike_out_netmob_POIs_calendar_late_f...  5.08 (-0.42\%)  0.060222   
STAEformer_bike_out_netmob_POIs_calendar_late_f...  5.08 (-0.34\%)  0.042590   
...                                                            ...       ...   
STAEformer_bike_out_netmob_POIs_calendar_late_f...   5.19 (1.88\%)  0.097070   
STAEformer_bike_out_netmob_POIs_calendar_late_f...    5.2 (2.07\%)  0.077401   
STAEformer_bike_out_netmob_POIs_calendar_late_f...   5.23 (2.57\%)  0.125360   
STAEformer_bike_out_netmob_POIs_calendar_late_f...   5.24 (2.88\%)  0.097010   
STAEformer_bike_out_netmob_POIs_calendar_late_f...   5.32 (4.47\%)  0.130209   

                                                            mae_h4            \
                                                              mean       std   
STAEformer_bike_out_netmob_POIs_calendar_late_f...  3.32 (-0.48\%)  0.018730   
STAEformer_bike_out_netmob_POIs_calendar_late_f...  3.32 (-0.32\%)  0.023689   
STAEformer_bike_out_netmob_POIs_calendar_late_f...  3.32 (-0.41\%)  0.011750   
STAEformer_bike_out_netmob_POIs_calendar_late_f...  3.32 (-0.29\%)  0.017754   
STAEformer_bike_out_netmob_POIs_calendar_late_f...  3.33 (-0.13\%)  0.017124   
...                                                            ...       ...   
STAEformer_bike_out_netmob_POIs_calendar_late_f...   3.38 (1.41\%)  0.042882   
STAEformer_bike_out_netmob_POIs_calendar_late_f...   3.37 (1.01\%)  0.036693   
STAEformer_bike_out_netmob_POIs_calendar_late_f...   3.39 (1.75\%)  0.059903   
STAEformer_bike_out_netmob_POIs_calendar_late_f...    3.4 (2.09\%)  0.050209   
STAEformer_bike_out_netmob_POIs_calendar_late_f...   3.45 (3.49\%)  0.070907   

                                                          mase_h4            
                                                             mean       std  
STAEformer_bike_out_netmob_POIs_calendar_late_f...  0.8 (-0.47\%)  0.004536  
STAEformer_bike_out_netmob_POIs_calendar_late_f...  0.8 (-0.33\%)  0.005607  
STAEformer_bike_out_netmob_POIs_calendar_late_f...   0.8 (-0.4\%)  0.002730  
STAEformer_bike_out_netmob_POIs_calendar_late_f...   0.8 (-0.3\%)  0.004144  
STAEformer_bike_out_netmob_POIs_calendar_late_f...  0.8 (-0.14\%)  0.004069  
...                                                           ...       ...  
STAEformer_bike_out_netmob_POIs_calendar_late_f...   0.81 (1.4\%)  0.010390  
STAEformer_bike_out_netmob_POIs_calendar_late_f...  0.81 (0.99\%)  0.008774  
STAEformer_bike_out_netmob_POIs_calendar_late_f...  0.82 (1.73\%)  0.014428  
STAEformer_bike_out_netmob_POIs_calendar_late_f...  0.82 (2.08\%)  0.012172  
STAEformer_bike_out_netmob_POIs_calendar_late_f...  0.83 (3.49\%)  0.017067  

[65 rows x 6 columns]

rmse_h1            \
                                                              mean       std   
STAEformer_bike_out_netmob_POIs_calendar_late_f...  4.52 (-0.47\%)  0.010090   
STAEformer_bike_out_netmob_POIs_calendar_late_f...   4.52 (-0.4\%)  0.022036   
STAEformer_bike_out_netmob_POIs_calendar_late_f...  4.52 (-0.38\%)  0.022683   
STAEformer_bike_out_netmob_POIs_calendar_late_f...  4.52 (-0.33\%)  0.020607   
STAEformer_bike_out_netmob_POIs_calendar_late_f...  4.53 (-0.27\%)  0.034003   
...                                                            ...       ...   
STAEformer_bike_out_netmob_POIs_calendar_late_f...   4.56 (0.48\%)  0.036897   
STAEformer_bike_out_netmob_POIs_calendar_late_f...   4.56 (0.51\%)  0.033563   
STAEformer_bike_out_netmob_POIs_calendar_late_f...   4.56 (0.52\%)  0.025341   
STAEformer_bike_out_netmob_POIs_calendar_late_f...   4.56 (0.59\%)  0.029278   
STAEformer_bike_out_netmob_POIs_calendar_late_f...    4.57 (0.8\%)  0.027669   

                                                            mae_h1            \
                                                              mean       std   
STAEformer_bike_out_netmob_POIs_calendar_late_f...  3.07 (-0.04\%)  0.003886   
STAEformer_bike_out_netmob_POIs_calendar_late_f...  3.06 (-0.42\%)  0.008239   
STAEformer_bike_out_netmob_POIs_calendar_late_f...  3.06 (-0.35\%)  0.008509   
STAEformer_bike_out_netmob_POIs_calendar_late_f...  3.06 (-0.33\%)  0.012357   
STAEformer_bike_out_netmob_POIs_calendar_late_f...  3.06 (-0.18\%)  0.013454   
...                                                            ...       ...   
STAEformer_bike_out_netmob_POIs_calendar_late_f...    3.07 (0.1\%)  0.016887   
STAEformer_bike_out_netmob_POIs_calendar_late_f...   3.07 (0.07\%)  0.012700   
STAEformer_bike_out_netmob_POIs_calendar_late_f...   3.07 (0.03\%)  0.009019   
STAEformer_bike_out_netmob_POIs_calendar_late_f...   3.07 (0.11\%)  0.012213   
STAEformer_bike_out_netmob_POIs_calendar_late_f...   3.08 (0.36\%)  0.007406   

                                                           mase_h1            
                                                              mean       std  
STAEformer_bike_out_netmob_POIs_calendar_late_f...  0.74 (-0.04\%)  0.000928  
STAEformer_bike_out_netmob_POIs_calendar_late_f...  0.73 (-0.42\%)  0.001977  
STAEformer_bike_out_netmob_POIs_calendar_late_f...  0.74 (-0.35\%)  0.002072  
STAEformer_bike_out_netmob_POIs_calendar_late_f...  0.74 (-0.33\%)  0.002970  
STAEformer_bike_out_netmob_POIs_calendar_late_f...  0.74 (-0.18\%)  0.003254  
...                                                            ...       ...  
STAEformer_bike_out_netmob_POIs_calendar_late_f...   0.74 (0.09\%)  0.004058  
STAEformer_bike_out_netmob_POIs_calendar_late_f...   0.74 (0.07\%)  0.003062  
STAEformer_bike_out_netmob_POIs_calendar_late_f...   0.74 (0.03\%)  0.002158  
STAEformer_bike_out_netmob_POIs_calendar_late_f...   0.74 (0.11\%)  0.002947  
STAEformer_bike_out_netmob_POIs_calendar_late_f...   0.74 (0.36\%)  0.001809  

[65 rows x 6 columns]


------------------ Exp6_subway_netmob ---


rmse_h4            \
                                                              mean       std   
STAEformer_subway_in_calendar__e150_h4               43.73 (0.0\%)  2.225578   
STAEformer_subway_in_netmob_POIs_calendar_late_...  44.25 (1.19\%)  2.999800   
STAEformer_subway_in_netmob_POIs_calendar_late_...  44.83 (2.51\%)  2.316863   
STAEformer_subway_in_netmob_POIs_calendar_late_...  45.37 (3.76\%)  0.988276   
STAEformer_subway_in_netmob_POIs_calendar_late_...  45.66 (4.42\%)  1.828776   
STAEformer_subway_in_netmob_POIs_calendar_late_...  45.92 (5.01\%)  2.109974   
STAEformer_subway_in_netmob_POIs_calendar_late_...   46.05 (5.3\%)  2.295588   
STAEformer_subway_in_netmob_POIs_calendar_late_...   46.09 (5.4\%)  2.408815   
STAEformer_subway_in_netmob_POIs_calendar_late_...  46.16 (5.56\%)  3.189010   

                                                            mae_h4            \
                                                              mean       std   
STAEformer_subway_in_calendar__e150_h4               25.33 (0.0\%)  1.145708   
STAEformer_subway_in_netmob_POIs_calendar_late_...  25.48 (0.57\%)  1.052312   
STAEformer_subway_in_netmob_POIs_calendar_late_...  25.41 (0.32\%)  0.930834   
STAEformer_subway_in_netmob_POIs_calendar_late_...  25.86 (2.11\%)  0.226085   
STAEformer_subway_in_netmob_POIs_calendar_late_...   25.79 (1.8\%)  0.771501   
STAEformer_subway_in_netmob_POIs_calendar_late_...  25.91 (2.28\%)  0.686228   
STAEformer_subway_in_netmob_POIs_calendar_late_...   25.96 (2.5\%)  0.917877   
STAEformer_subway_in_netmob_POIs_calendar_late_...  26.37 (4.12\%)  1.016179   
STAEformer_subway_in_netmob_POIs_calendar_late_...  26.21 (3.46\%)  0.936478   

                                                          mase_h4            
                                                             mean       std  
STAEformer_subway_in_calendar__e150_h4               0.81 (0.0\%)  0.036620  
STAEformer_subway_in_netmob_POIs_calendar_late_...  0.82 (0.57\%)  0.033686  
STAEformer_subway_in_netmob_POIs_calendar_late_...  0.81 (0.32\%)  0.029798  
STAEformer_subway_in_netmob_POIs_calendar_late_...  0.83 (2.11\%)  0.007292  
STAEformer_subway_in_netmob_POIs_calendar_late_...   0.83 (1.8\%)  0.024721  
STAEformer_subway_in_netmob_POIs_calendar_late_...  0.83 (2.28\%)  0.021967  
STAEformer_subway_in_netmob_POIs_calendar_late_...   0.83 (2.5\%)  0.029370  
STAEformer_subway_in_netmob_POIs_calendar_late_...  0.84 (4.12\%)  0.032476  
STAEformer_subway_in_netmob_POIs_calendar_late_...  0.84 (3.46\%)  0.029944

rmse_h1            \
                                                               mean       std   
STAEformer_subway_in_netmob_POIs_calendar_late_...  35.74 (-1.71\%)  0.711533   
STAEformer_subway_in_netmob_POIs_calendar_late_...  35.93 (-1.18\%)  0.716963   
STAEformer_subway_in_netmob_POIs_calendar_late_...  35.94 (-1.16\%)  1.445708   
STAEformer_subway_in_netmob_POIs_calendar_late_...  36.03 (-0.91\%)  0.826459   
STAEformer_subway_in_netmob_POIs_calendar_late_...  36.22 (-0.37\%)  0.929231   
STAEformer_subway_in_netmob_POIs_calendar_late_...  36.29 (-0.19\%)  0.294127   
STAEformer_subway_in_calendar__e150_h1                36.36 (0.0\%)  0.416060   
STAEformer_subway_in_netmob_POIs_calendar_late_...   36.46 (0.28\%)  1.345812   
STAEformer_subway_in_netmob_POIs_calendar_late_...    36.6 (0.68\%)  0.759024   

                                                             mae_h1            \
                                                               mean       std   
STAEformer_subway_in_netmob_POIs_calendar_late_...   21.2 (-1.52\%)  0.300492   
STAEformer_subway_in_netmob_POIs_calendar_late_...  21.26 (-1.21\%)  0.249830   
STAEformer_subway_in_netmob_POIs_calendar_late_...  21.22 (-1.39\%)  0.729602   
STAEformer_subway_in_netmob_POIs_calendar_late_...  21.33 (-0.89\%)  0.503613   
STAEformer_subway_in_netmob_POIs_calendar_late_...  21.45 (-0.32\%)  0.587913   
STAEformer_subway_in_netmob_POIs_calendar_late_...    21.5 (-0.1\%)  0.212985   
STAEformer_subway_in_calendar__e150_h1                21.52 (0.0\%)  0.251448   
STAEformer_subway_in_netmob_POIs_calendar_late_...   21.54 (0.08\%)  0.688813   
STAEformer_subway_in_netmob_POIs_calendar_late_...   21.68 (0.71\%)  0.454884   

                                                           mase_h1            
                                                              mean       std  
STAEformer_subway_in_netmob_POIs_calendar_late_...  0.68 (-1.52\%)  0.009615  
STAEformer_subway_in_netmob_POIs_calendar_late_...  0.68 (-1.22\%)  0.007971  
STAEformer_subway_in_netmob_POIs_calendar_late_...   0.68 (-1.4\%)  0.023354  
STAEformer_subway_in_netmob_POIs_calendar_late_...  0.68 (-0.89\%)  0.016109  
STAEformer_subway_in_netmob_POIs_calendar_late_...  0.69 (-0.32\%)  0.018871  
STAEformer_subway_in_netmob_POIs_calendar_late_...   0.69 (-0.1\%)  0.006869  
STAEformer_subway_in_calendar__e150_h1                0.69 (0.0\%)  0.008054  
STAEformer_subway_in_netmob_POIs_calendar_late_...   0.69 (0.09\%)  0.022065  
STAEformer_subway_in_netmob_POIs_calendar_late_...   0.69 (0.71\%)  0.014583

In [7]:

def change_legend(row):
    if row.legend_group == 'Other Methods':
        if 'early_fusion' in row.name:
            return 'Early Fusion Other Methods '
        elif 'late_fusion' in row.name:
            return 'Late Fusion Other Methods '
        else:
            return row.legend_group
    else:
        return row.legend_group

def reduce_id(x):
    x = x.replace('late_fusion_','L_')
    x = x.replace('early_fusion_','E_')
    x = x.replace('backbone','BB')
    return x 


# configs_to_keep = [
#     'Baseline',
#     # Model backbone : ---- 
#     'late_fusion_traffic_model_backbone_In24_adp16',
#     'early_fusion_traffic_model_backbone_In24_adp16',
#     # CABB with adpQ = 0 ---
#     'early_fusion_CABB_In24_ctxIn8_adp16_adpQ0',
#     'late_fusion_CABB_In24_ctxIn8_adp16_adpQ0',
#     # CABB with adpQ = 24 ---
#     'early_fusion_CABB_In24_ctxIn8_adp16_adpQ24',
#     'late_fusion_CABB_In24_ctxIn8_adp16_adpQ24',
#     # 'early_fusion_CABB_In24_ctxIn24_adp16_adpQ24',
#     # 'late_fusion_CABB_In24_ctxIn24_adp16_adpQ24',
    
#     # Others ---
#     'late_fusion_simple_embedding',
#     'early_fusion_shared_embedding',
#     'early_fusion_independant_embedding'   
# ] 
configs_to_keep = None 

# palette = ['#4e79a7','#f28e2b','#e15759', '#59a14f', '#c7e9c0','#1a4314' ]
# legend_groups = ['Baseline','Independant Embedding','Shared Embedding','Early Fusion Other Methods ','Late Fusion Other Methods ']
palette = None
legend_groups = None
metrics = ['rmse','mae','mase']

# -------------------------------------------------- 
# --------------------------------------------------        




experiences = ['Exp6_bike_netmob','Exp6_subway_netmob'] #['Exp3','Exp3_bike_15min_h4','Exp4','Exp4_15min_h1','Exp4_15min']
for exp_i in experiences:

    trials = dic_trials[exp_i]
    print(f"\n------------------ {exp_i} ---")

    L_metrics = []
    for trial_j in trials:
        if (exp_i == 'Exp3_bike_15min_h4') and ('STAEformer_bike_out_calendar__e50_h4' in trial_j or 'STAEformer_bike_out_calendar__e80_h4' in trial_j):
            exp_i_tmp = 'Exp4_15min'
            # trial_j = 'STAEformer_bike_out_calendar__e200_h4'
        elif exp_i == 'Exp4_15min_h1':
            exp_i_tmp = 'Exp4_15min'
        else:
            exp_i_tmp = exp_i
        L_metrics = tackle_trial_for_distrib(folder_path,dic_exp_to_names,L_metrics,exp_i_tmp,trial_j,metrics)


    df_metrics_all = pd.concat(L_metrics)

    horizons = list(set([c.split('_')[-1][1:] for c in df_metrics_all.columns]))
    for horizon in horizons:
        print('\n----------------')
        print(f"Horizon: {horizon}")
        df_horizon = df_metrics_all[[c for c in df_metrics_all.columns if c.endswith(f"_h{horizon}")]].dropna()

        df_horizon = update_df_metrics(df_horizon,exp_i)
        df_horizon['legend_group'] = df_horizon.apply(change_legend,axis = 1)

        if configs_to_keep is not None: 
            df_horizon = df_horizon[df_horizon['id'].isin(configs_to_keep)]
            df_horizon['id'] = df_horizon['id'].apply(reduce_id)

        plot_boxplot_on_metric(df_horizon, metric_i='mae', xaxis_label="Config", legend_group='legend_group', width=800, height=600, 
                                    save_path=None,palette =palette ,legend_groups =legend_groups )
        plot_boxplot_on_metric(df_horizon, metric_i='rmse', xaxis_label="Config", legend_group='legend_group', width=800, height=600, 
                                    save_path=None,palette =palette ,legend_groups =legend_groups)
        # plot_boxplot_on_metric(df_horizon, metric_i='mase', xaxis_label="Config", legend_group='legend_group', width=1200, height=400, 
        #                             save_path=None)
    


------------------ Exp6_bike_netmob ---

----------------
Horizon: 4


Loading BokehJS ...

Loading BokehJS ...


----------------
Horizon: 1


Loading BokehJS ...

...

Loading BokehJS ...

Loading BokehJS ...


----------------
Horizon: 1


Loading BokehJS ...

Loading BokehJS ...

In [5]:
from experiences.contextual_data_integration.A_heterogenous.exp4_15min_h1_results import results as results_Exp4_15min_h1
from experiences.contextual_data_integration.A_heterogenous.exp4_15min_results import results  as results_Exp4_15min


def results_to_dataframe(results,get_id = False):
    """
    Cette fonction transforme les résultats bruts en un DataFrame pandas.
    """
    data = []
    columns=['target', 'contextual_data', 'percentage','horizon', 'bis', 'RMSE', 'MAE', 'MASE']
    if get_id:
        columns.insert(2,'id')
    for line in results.strip().split('\n'):
        if not line.strip():
            continue
        
        parts = line.split(':')
        name = parts[0].strip()
        metrics = parts[1].strip()
        
        target_match = re.search(r'(bike_out|subway_out)', name)
        target = target_match.group(1) if target_match else 'unknown'
        
        contextual_data = target
        if 'weather' in name and 'subway_in' in name:
            contextual_data = f"{target} + subway-in + weather"
        elif 'weather' in name:
            contextual_data = f"{target}_weather"
        elif 'subway_in' in name:
            contextual_data = f"{target} + subway in"

        # If startwith <model_name>_<target>_calendar_ : then id = baseline 
        # Else: id is the string between calendar (or calendar_embedding) and __e(\d)_h(\d)_bis(\d). Like <model_name>_<target>_calendar_<id>__e(\d)_h(\d)_bis(\d)
        # print(name)
        # print(line)
        id_match = re.search(r'calendar_(.*?)__e', name)
        if id_match:
            # print(id_match.group(1))
            t_match = re.search(r'_t(\d)',name)
            if t_match:
                print(id_match.group(1))
                t_value = t_match.group(1)
                id_str = f'{id_match.group(1)}_{t_value}'
            else:
                id_str = id_match.group(1) 
        else :
            t_match = re.search(r'_t(\d)',name)
            if t_match:
                t_value = t_match.group(1)
                id_str = f'baseline_{t_value}'
            else:
                id_str = 'baseline'
            
            'baseline'
        
        percentage_match = re.search(r'(\d+)p', name) 
        percentage = int(percentage_match.group(1)) if percentage_match else -1
        
        horizon_match = re.search(r'_h(\d+)_', name)
        horizon = int(horizon_match.group(1)) if horizon_match else -1

        bis_match = re.search(r'bis(\d)', name)
        bis = int(bis_match.group(1)) if bis_match else -1

        rmse_match = re.search(r'RMSE = ([\d.]+)', metrics)
        rmse = float(rmse_match.group(1)) if rmse_match else -1.0
        
        mae_match = re.search(r'MAE = ([\d.]+)', metrics)
        mae = float(mae_match.group(1)) if mae_match else -1.0
        
        mase_match = re.search(r'MASE = ([\d.]+)', metrics)
        mase = float(mase_match.group(1)) if mase_match else -1.0
        if get_id:
            data.append([target, contextual_data, id_str, percentage, horizon, bis, rmse, mae, mase])
        else:
            data.append([target, contextual_data, percentage, horizon, bis, rmse, mae, mase])
        
    df = pd.DataFrame(data,columns = columns )
    return df


results = results_Exp4_15min_h1 # results_Exp4_15min # results_Exp4_15min_h1
df_metric_all = results_to_dataframe(results,get_id = True)
df_metric_all.id = df_metric_all.id.apply(lambda x: x.replace('late_fusion','L'))
df_metric_all.id = df_metric_all.id.apply(lambda x: x.replace('early_fusion','E'))
df_metric_all.id = df_metric_all.id.apply(lambda x: x.replace('CrossAttnBackBone','CABB'))
df_metric_all.id = df_metric_all.id.apply(lambda x: x.replace('ctxInEmb','ctx'))
df_metric_all.id = df_metric_all.id.apply(lambda x: x.replace('InEmb','In'))
df_metric_all.id = df_metric_all.id.apply(lambda x: x.replace('adpQ','Q'))
display(df_metric_all.head())

def add_legend_group(row):
    if 'baseline' in row['id']:
        return 'Baseline'
    elif 'L_CABB' in row['id']:
        return 'Late Fusion CABB'
    elif 'E_CABB' in row['id']:
        return 'Early Fusion CABB'
    else:
        return 'Other Methods'

configs_to_keep = None
horizons = df_metric_all['horizon'].unique()
for horizon in horizons:
    print('\n----------------')
    print(f"Horizon: {horizon}")
    df_horizon = df_metric_all[df_metric_all['horizon'] == horizon]
    df_horizon['legend_group'] = df_horizon.apply(add_legend_group,axis = 1)

    if configs_to_keep is not None: 
        df_horizon = df_horizon[df_horizon['id'].isin(configs_to_keep)]
        df_horizon['id'] = df_horizon['id'].apply(reduce_id)

    df_horizon = df_horizon.rename(columns ={'MAE':'mae','RMSE':'rmse','MASE':'mase'})

    plot_boxplot_on_metric(df_horizon, metric_i='mae', xaxis_label="Config", legend_group='legend_group', width=800, height=600, 
                                save_path=None,palette =palette ,legend_groups =legend_groups )
    plot_boxplot_on_metric(df_horizon, metric_i='rmse', xaxis_label="Config", legend_group='legend_group', width=800, height=600, 
                                save_path=None,palette =palette ,legend_groups =legend_groups)

late_fusion_CrossAttnBackBone_InEmb24_ctxInEmb8_adp16_adpQ0
late_fusion_CrossAttnBackBone_InEmb24_ctxInEmb8_adp16_adpQ0
late_fusion_CrossAttnBackBone_InEmb24_ctxInEmb8_adp16_adpQ0
late_fusion_CrossAttnBackBone_InEmb24_ctxInEmb8_adp16_adpQ0
late_fusion_CrossAttnBackBone_InEmb24_ctxInEmb8_adp16_adpQ0


,target,contextual_data,id,percentage,horizon,bis,RMSE,MAE,MASE
0,bike_out,bike_out,baseline_2,-1,1,1,4.309,2.810,0.730
1,bike_out,bike_out,baseline_2,-1,1,2,4.319,2.784,0.723
2,bike_out,bike_out,baseline_2,-1,1,3,4.284,2.780,0.722
3,bike_out,bike_out,baseline_2,-1,1,4,4.276,2.774,0.720
4,bike_out,bike_out,baseline_2,-1,1,5,4.340,2.794,0.726



----------------
Horizon: 1


Loading BokehJS ...

Loading BokehJS ...

In [ ]:
# --- Captions Dictionary ---
captions = {
    "Exp1": r"Prediction of Subway-Out with STAEformer using contextual data 'Subway-In' according to different Integration Strategies. Metrics are averaged over 5 runs.",
    "Exp1_subway_in": r"Prediction of Subway-In with STAEformer using contextual data 'Subway-Out' according to different Integration Strategies. Metrics are averaged over 5 runs.",
    "Exp2": r"Prediction of Bike-Out (15min aggregation) at a 60-minute horizon with STAEformer, using weather forecasts as contextual data. Metrics are averaged over 5 runs.",
    "Exp4": r"Prediction of Bike-Out with STAEformer using heterogenous contextual data 'subway-in-subway-out' (1h aggregation) at horizon +60min. The table compares the baseline with different configurations of the CrossAttnBackBone model. Metrics are averaged over 5 runs.",
    "Exp4_15min": r"Prediction of Bike-Out with STAEformer using heterogenous contextual data 'subway-in-subway-out' (15min aggregation) at horizon +60min. This table compares various CrossAttnBackBone model configurations. Metrics are averaged over 5 runs.",
     "Exp4_15min_h1": r"Prediction of Bike-Out with STAEformer using heterogenous contextual data 'subway-in-subway-out' (15min aggregation) at horizon +15min. This table compares various CrossAttnBackBone model configurations. Metrics are averaged over 5 runs.",
}

# --- Generate and Print LaTeX Tables ---
print("%%% LaTeX code for Experiment 1 Table %%%\n")
latex_table_1 = dataframe_to_latex(df_metrics_Exp1_h4, captions["Exp1"], "exp1_subway_context", parse_index_exp1_2)
print(latex_table_1)

print("\n" + "="*80 + "\n")

print("\n------------------------------------------------\n%%% LaTeX code for Experiment 1 Horizon +15min Table %%%\n")
latex_table_1 = dataframe_to_latex(df_metrics_Exp1_subway_in_h1, captions["Exp1_subway_in"], "exp1_subway_context", lambda x: parse_index_exp1_2(x, contextual='subway_out'))
print(latex_table_1)

print("\n" + "="*80 + "\n")

print("\n------------------------------------------------\n%%% LaTeX code for Experiment 1 Horizon +60min Table %%%\n")
latex_table_1 = dataframe_to_latex(df_metrics_Exp1_subway_in_h4, captions["Exp1_subway_in"], "exp1_subway_context", lambda x: parse_index_exp1_2(x, contextual='subway_out'))
print(latex_table_1)

print("\n" + "="*80 + "\n")

print("\n------------------------------------------------\n%%% LaTeX code for Experiment 2 Table %%%\n")
# Note: parse_index_exp1_2 is reused as it correctly handles the index format for Exp2
latex_table_2 = dataframe_to_latex(df_metrics_Exp2_h4, captions["Exp2"], "exp2_bike_weather", parse_index_exp1_2)
print(latex_table_2)

print("\n" + "="*80 + "\n")

print("\n------------------------------------------------\n%%% LaTeX code for Experiment 4 Table %%%\n")
latex_table_4 = dataframe_to_latex(df_metrics_Exp4_h1, captions["Exp4"], "exp4_bike_hetero", parse_index_exp4)
print(latex_table_4)

print("\n" + "="*80 + "\n")

print("\n------------------------------------------------\n%%% LaTeX code for Experiment 4 (15min) at horizon +60min Table %%%\n")
# Note: parse_index_exp4 is reused as it correctly handles the index format for Exp4_15min
latex_table_4_15min = dataframe_to_latex(df_metrics_Exp4_15min_h4, captions["Exp4_15min"], "exp4_bike_hetero_15min", parse_index_exp4)
print(latex_table_4_15min)

print("\n" + "="*80 + "\n")

print("\n------------------------------------------------\n%%% LaTeX code for Experiment 4 (agg 15min) at horizon + 15min Table %%%\n")
latex_table_4_15min_h1 = dataframe_to_latex(df_metrics_Exp4_15min_h4, captions["Exp4_15min_h1"], "exp4_bike_hetero_15min", parse_index_exp4)
print(latex_table_4_15min_h1)

print("\n" + "="*80 + "\n")


%%% LaTeX code for Experiment 1 Table %%%

\begin{table}[!htb]
    \centering
    \caption{Prediction of Subway-Out with STAEformer using contextual data 'Subway-In' according to different Integration Strategies. Metrics are averaged over 5 runs.}
    \label{tab:exp1_subway_context}
    \resizebox{\textwidth}{!}{
    \begin{tabular}{{llcccccc}}
        \toprule
        Contextual Data & Integration Strategy & \multicolumn{2}{c}{ RMSE (h4) } & \multicolumn{2}{c}{ MAE (h4) } & \multicolumn{2}{c}{ MASE (h4) } \\
        \cmidrule(lr){1-2} \cmidrule(lr){3-4} \cmidrule(lr){5-6} \cmidrule(lr){7-8}
         &  & Mean & Std & Mean & Std & Mean & Std \\
        \midrule
        Yes & Early Fusion Feature Extractor & 40.46 (-5.3\%) & 0.1824 & 22.27 (-4.02\%) & 0.0953 & 0.69 (-3.53\%) & 0.0030 \\
        Yes & Early Fusion Shared Embedding & 40.76 (-4.61\%) & 0.4171 & 22.3 (-3.88\%) & 0.2049 & 0.69 (-3.38\%) & 0.0064 \\
        Yes & Late Fusion Traffic Model Backbone & 40.8 (-4.52\%) & 0.2193 & 

In [22]:

import os 
import sys 

current_path = os.path.abspath(os.getcwd())
parent_dir = os.path.abspath(os.path.join(current_path, '..','..','..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.accuracy_comparison import load_trainer_ds_from_2_trials,get_predict_real_and_inputs,get_model_args
from examples.accuracy_comparison import get_rainy_indices,plot_analysis_comparison_2_config,get_previous_and_prediction,get_gain_from_mod1
import torch 
from pipeline.utils.metrics import evaluate_metrics

def get_dict_metrics_on_rainy_events(full_predict1,full_predict2,Y_true,X,args_init1,args_init2,ds2):
    h_idx = 1
    metric_list = ['rmse','mse','mae','mase','mape']
    previous,_,_,_ = get_previous_and_prediction(full_predict1,full_predict2,Y_true,X,h_idx)
    assert args_init1.horizon_step == args_init1.step_ahead, "Horizon step must be equal to step_ahead here"

    print("\nComparison on between models across all time-slots followed by comparison on Rainy Events Only")
    _,train_rainy_indices,_ = get_rainy_indices(args = args_init2,ds = ds2,training_mode = 'train')
    print(f"Number of rainy time-slots in the train set: {len(train_rainy_indices)}, i.e {len(train_rainy_indices)/len(ds2.tensor_limits_keeper.df_verif_train)*100:.2f} % of the train set")
    # ---- Plot Accuracy comparison on rainy moments only ----
    mask,rainy_indices,df_weather = get_rainy_indices(args = args_init2,ds = ds2,training_mode = 'test')
    print(f"Number of rainy time-slots in the test set: {len(rainy_indices)}, i.e {len(rainy_indices)/len(ds2.tensor_limits_keeper.df_verif_test)*100:.2f} % of the test set\n")

    dates = mask[mask].index
    masked_index = mask.reset_index(drop=True)
    masked_index = masked_index[masked_index].index


    # --- Get Prediction on rainy time-slots:
    rainy_predict1 = torch.index_select(full_predict1,0,torch.tensor(masked_index).long())
    rainy_predict2 = torch.index_select(full_predict2,0,torch.tensor(masked_index).long())
    rainy_Y_true = torch.index_select(Y_true,0,torch.tensor(masked_index).long())
    rainy_previous = torch.index_select(previous,0,torch.tensor(masked_index).long())

    dic_metric1 = evaluate_metrics(rainy_predict1,rainy_Y_true,metrics = metric_list, previous = rainy_previous,horizon_step = h_idx)
    dic_metric1 = dict(dic_metric1.copy())
    dic_metric2 = evaluate_metrics(rainy_predict2,rainy_Y_true,metrics = metric_list, previous = rainy_previous,horizon_step = h_idx)
    dic_metric2 = dict(dic_metric2.copy())
    return dic_metric1,dic_metric2



dic_contextual_data = {'bike_out': [[]],  #,['weather','calendar']
                    }

dic_fusion_strategie = {('bike_out',()): [''],
                        ('bike_out',('weather','calendar')):['early_fusion','late_fusion'],
                        }
             
dic_feature_extractor = {('bike_out',()): [''],
                        ('bike_out',('weather','calendar')):['repeat_t_proj','feature_extractor']
                        }


model_name = 'STAEformer'
epochs = 200
range_k = range(1,6)
training_mode = 'test'
modification = {'shuffle':False,
                'data_augmentation':False,
                'torch_compile': False,
                }
log = ''
Exp = 'Exp2'  # 'Exp2'

for horizon in [4]:
    for target in ['bike_out']:
        for contextual_data in dic_contextual_data[target]:
            reversed_metric = False
            fusion_strategies = dic_fusion_strategie[(target,tuple(contextual_data))]
            for fusion_strategie in fusion_strategies:
                feature_extractors = dic_feature_extractor[(target,tuple(contextual_data))]
                for feature_extractor in feature_extractors:
                    if not('weather' in contextual_data):
                        print('\n------------------------------------------------------\nContextual data:\n')
                        contextual_data1 = ['weather','calendar'] if target == 'bike_out' else ['subway_in','weather','calendar']
                        fusion_strategie1 = dic_fusion_strategie[(target,tuple(contextual_data1))][0]
                        feature_extractor1 = dic_feature_extractor[(target,tuple(contextual_data1))][0]

                        trial_id1 = f"{model_name}_{target}_calendar__e{epochs}_h{horizon}_bis"
                        trial_id2 = f"{model_name}_{target}_{'_'.join(contextual_data1)}_{fusion_strategie1}_{feature_extractor1}__e{epochs}_h{horizon}_bis"
                        reversed_metric = True 
                    else:
                        trial_id1 = f"{model_name}_{target}_calendar__e{epochs}_h{horizon}_bis"
                        trial_id2 = f"{model_name}_{target}_{'_'.join(contextual_data)}_{fusion_strategie}_{feature_extractor}__e{epochs}_h{horizon}_bis"

                    model_args,_,path_model_args,_ = get_model_args(target,model_name,save_folder_name = f'{Exp}/{target}_{model_name}')
                    ds1,ds2,args_init1,args_init2 = None, None, None, None
                    for k in range_k:
                        trial_id1_updated = f"{trial_id1}{k}_f5"
                        trial_id2_updated = f"{trial_id2}{k}_f5"

                        trainer1,trainer2,ds1,ds2,args_init1,args_init2 = load_trainer_ds_from_2_trials(trial_id1_updated,trial_id2_updated,modification = modification,
                                                                                                        model_args=model_args,
                                                                                                        path_model_args=path_model_args,
                                                                                                        ds1_init=ds1,ds2_init=ds2,
                                                                                                        args_init1=args_init1,args_init2=args_init2,
                                                                                                        )
                                                                                                        

                        full_predict1,full_predict2,Y_true,X = get_predict_real_and_inputs(trainer1,trainer2,ds1,ds2,training_mode=training_mode)

                        globals()[f"trainer1_bis{k}"] = trainer1
                        globals()[f"trainer2_bis{k}"] = trainer2
                        globals()[f"ds1_bis{k}"] = ds1
                        globals()[f"ds2_bis{k}"] = ds2
                        globals()[f"full_predict1_bis{k}"] = full_predict1
                        globals()[f"full_predict2_bis{k}"] = full_predict2

                        dic_metric1,dic_metric2 = get_dict_metrics_on_rainy_events(globals()[f"full_predict1_bis{k}"],globals()[f"full_predict2_bis{k}"],Y_true,X,args_init1,args_init2,ds2)

                        # Keep track on metric from model 1
                        if reversed_metric:
                            dic_metric2 = dict(dic_metric1.copy())
                        RMSE = dic_metric2['rmse_all']
                        MAE = dic_metric2['mae_all']
                        MASE = dic_metric2['mase_all']
                        MAPE = dic_metric2['mape_all']
                        if feature_extractor == []:
                            log += f"{model_name}_{target}__e{epochs}_h{horizon}_bis{k}:   All Steps RMSE = {RMSE:.3f}, MAE = {MAE:.3f}, MASE = {MASE:.3f}, MAPE = {MAPE:.3f}\n"
                        else:
                            log += f"{model_name}_{target}_{'_'.join(contextual_data)}_{fusion_strategie}_{feature_extractor}__e{epochs}_h{horizon}_bis{k}:   All Steps RMSE = {RMSE:.3f}, MAE = {MAE:.3f}, MASE = {MASE:.3f}, MAPE = {MAPE:.3f}\n"

                    print(log)


------------------------------------------------------
Contextual data:

Trial ID 1:  STAEformer_bike_out_calendar__e200_h4_bis1_f5
Trial ID 2:  STAEformer_bike_out_weather_calendar_early_fusion_repeat_t_proj__e200_h4_bis1_f5
model_save_path for trial id1:  /home/rrochas/prediction-validation/save/K_fold_validation/training_wo_HP_tuning/Exp2/bike_out_STAEformer/best_models/STAEformer_bike_out_calendar__e200_h4_bis1_f5.pkl
----------------------------------------
Loading the Complete Dataset for K-fold splitting
Coverage Period: 35040 elts between 2019-01-01 00:00:00 and 2019-12-31 23:45:00
Invalid dates within this fold: 0

>>>Tackle Target dataset: bike_out
Loading from /home/rrochas/../../data/rrochas/prediction_validation/agg_data/velov/velov_emitted_by_station15min.csv...
df pivoted:  (70049, 434)
df reindexed :  (35040, 434)
Len coverage period:  35040
df filtered:  (35040, 434)
   Loaded data: (35040, 434)
   Dimension after spatial agg: (35040, 26)
   Init Dataset: 'torch.Size(